# Analysis of Rotated_II meteo files

In [ ]:
#to use the full width of the browser window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
from pyresample import bilinear, geometry, kd_tree
from pyresample import utils
import xarray as xr

In [ ]:
%matplotlib notebook

In [ ]:
filename= '/Users/brey/Downloads/HNMS/01/E_JRC0000lf0000000020180101'

In [ ]:
filename= '/Users/brey/Downloads/HNMS/E_JRC0000lf0000000020170107'

In [ ]:
filename = '/Users/brey/Downloads/AM/JRC_0302_00_000'

# With pygrib

In [ ]:
import pygrib

In [ ]:
d=pygrib.open(filename)

In [ ]:
for m in d:
    print m

In [ ]:
v = d.message(3)

In [ ]:
v.keys()

In [ ]:
v['shortName']

In [ ]:
for key in v.keys():
    print key, v[key]

In [ ]:
lon = v['longitudes']
lat = v['latitudes']
q = v.values

In [ ]:
v['gridType']

In [ ]:
lon = lon.reshape(q.shape)
lat = lat.reshape(q.shape)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable


import cartopy.crs as ccrs

proj=ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=(12, 8))

ax = plt.axes(projection=proj)

h = ax.contourf(lon,lat,q, transform=proj);

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')

divider = make_axes_locatable(ax)
ax_cb = divider.new_horizontal(size="5%", pad=0.5, axes_class=plt.Axes)


fig.add_axes(ax_cb)
plt.colorbar(h, cax=ax_cb)
#ax.set_global(); 
ax.coastlines();

# ...reading with eccodes

In [ ]:
from gribapi import *

In [ ]:
f=open(filename)
gid = grib_new_from_file(f)

In [ ]:
iterid = grib_keys_iterator_new(gid,'ls')
while grib_keys_iterator_next(iterid):
    keyname = grib_keys_iterator_get_name(iterid)
    keyval = grib_get_string(iterid,keyname)
    print "%s = %s" % (keyname,keyval)
    
grib_keys_iterator_delete(iterid)
grib_release(gid)
 
f.close()


In [ ]:
keys={'Ni',
'Nj',
'angleOfRotationInDegrees',
'iDirectionIncrementInDegrees',
'iScansNegatively',
'jDirectionIncrementInDegrees',
'jPointsAreConsecutive',
'jScansPositively',
'latitudeOfFirstGridPointInDegrees',
'latitudeOfLastGridPointInDegrees',
'latitudeOfSouthernPoleInDegrees',
'longitudeOfFirstGridPointInDegrees',
'longitudeOfLastGridPointInDegrees',
'longitudeOfSouthernPoleInDegrees'}

In [ ]:
f=open(filename)
gid = grib_new_from_file(f)
for key in keys:
        if not grib_is_defined(gid, key):
            raise ValueError("Key '%s' was not defined" % key)
        print('%s=%s' % (key, grib_get(gid, key)))
 


In [ ]:
print grib_get(gid, 'shortName')
print grib_get(gid,'missingValue')

print grib_get(gid, 'date')
print grib_get(gid, 'dataTime')
print grib_get(gid, 'stepRange')

print grib_get(gid,'longitudeOfFirstGridPointInDegrees')
print grib_get(gid,'latitudeOfFirstGridPointInDegrees')
print grib_get(gid,'longitudeOfLastGridPointInDegrees')
print grib_get(gid,'latitudeOfLastGridPointInDegrees')
print grib_get(gid,'gridType')

Ni=grib_get(gid,'Ni')
Nj=grib_get(gid,'Nj')


values=grib_get_values(gid)
elat=grib_get_array(gid,'latitudes')
elon=grib_get_array(gid,'longitudes')

In [ ]:
lat0 = grib_get(gid,'latitudeOfSouthernPoleInDegrees')
lon0 = grib_get(gid,'longitudeOfSouthernPoleInDegrees')
print lat0,lon0

In [ ]:
Ni,Nj

In [ ]:
elon = elon.reshape(Nj,Ni)
elat = elat.reshape(Nj,Ni)
q = values.reshape(Nj,Ni)

## Resample on rectangular grid

In [ ]:
import pyproj
prj = pyproj.Proj('+proj=eqc +lat_ts=0 +lat_0=0 +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs')
[[a0,a1],[a2,a3]] = prj([-7, 29.], [43., 48])
a0,a1,a2,a3

In [ ]:
orig = geometry.SwathDefinition(lons=elon,lats=elat) # original points

In [ ]:
area_id = 'HNMS'
description = 'HNMS COSMO'
proj_id = 'eqc'
projection = '+proj=eqc +lat_ts=0 +lat_0=0 +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m'
x_size = Ni
y_size = Nj
area_extent = (a0, a1, a2, a3)
target_def = utils.get_area_def(area_id, description, proj_id, projection,
                                       x_size, y_size, area_extent)
print(target_def)


In [ ]:
plons,plats = geometry.AreaDefinition.get_lonlats(target_def)

#### Nearest

In [ ]:
#kd_tree.resample_nearest(orig,q,targ,radius_of_influence=50000,fill_value=999999)
near = kd_tree.resample_nearest(orig, q, target_def, radius_of_influence=50000, epsilon=0.5)

In [ ]:
#w=np.ma.masked_array(iu10,iu10==999999.)

In [ ]:
xv=[q,near]
xp=[elon,plons]
yp=[elat,plats]

In [ ]:
import matplotlib.pyplot as plt

import cartopy.crs as ccrs

from mpl_toolkits.axes_grid1 import make_axes_locatable
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from mpl_toolkits.axes_grid1 import AxesGrid

projection = ccrs.PlateCarree()
axes_class = (GeoAxes,
                  dict(map_projection=projection))

fig = plt.figure(figsize=(12,8))
axgr = AxesGrid(fig, 111, axes_class=axes_class,
                    nrows_ncols=(1, 2),
                    axes_pad=1,
                    cbar_location='right',
                    cbar_mode='single',
                    cbar_pad=0.5,
                    cbar_size='10%',
                    label_mode='')  # note the empty label_mode


for i, ax in enumerate(axgr):
        ax.coastlines()
        ax.set_xticks(np.linspace(-180, 180, 5), crs=projection)
        ax.set_yticks(np.linspace(-90, 90, 5), crs=projection)
        lon_formatter = LongitudeFormatter(zero_direction_label=True)
        lat_formatter = LatitudeFormatter()
        ax.xaxis.set_major_formatter(lon_formatter)
        ax.yaxis.set_major_formatter(lat_formatter)

        p = ax.contourf(xp[i], yp[i],xv[i],
                        transform=projection)#,
                     #   cmap='RdBu')

axgr.cbar_axes[0].colorbar(p)




#### bilinear

In [ ]:
#%%timeit
blinear = bilinear.resample_bilinear(q, orig, target_def,
                                      radius=50e3, neighbours=32,
                                      nprocs=4, fill_value=None,
                                      reduce_data=False, segments=None,
                                      epsilon=0)

In [ ]:
blinear

In [ ]:
t_params, s_params, input_idxs, idx_ref = bilinear.get_bil_info(orig, target_def, radius=50e3, nprocs=1)
res = bilinear.get_sample_from_bil_info(q.ravel(), t_params, s_params,
                                          input_idxs, idx_ref,
                                          output_shape=target_def.shape)

In [ ]:
xv=[q,blinear,res]
xp=[elon,plons,plons]
yp=[elat,plats,plats]

In [ ]:
import matplotlib.pyplot as plt

import cartopy.crs as ccrs

from mpl_toolkits.axes_grid1 import make_axes_locatable
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from mpl_toolkits.axes_grid1 import AxesGrid

projection = ccrs.PlateCarree()
axes_class = (GeoAxes,
                  dict(map_projection=projection))

fig = plt.figure(figsize=(12,5))
axgr = AxesGrid(fig, 111, axes_class=axes_class,
                    nrows_ncols=(1, 3),
                    axes_pad=1,
                    cbar_location='right',
                    cbar_mode='single',
                    cbar_pad=0.5,
                    cbar_size='10%',
                    label_mode='')  # note the empty label_mode


for i, ax in enumerate(axgr):
        ax.coastlines()
        ax.set_xticks(np.linspace(-180, 180, 5), crs=projection)
        ax.set_yticks(np.linspace(-90, 90, 5), crs=projection)
        lon_formatter = LongitudeFormatter(zero_direction_label=True)
        lat_formatter = LatitudeFormatter()
        ax.xaxis.set_major_formatter(lon_formatter)
        ax.yaxis.set_major_formatter(lat_formatter)

        p = ax.contourf(xp[i], yp[i],xv[i],
                        transform=projection)#,
                     #   cmap='RdBu')

axgr.cbar_axes[0].colorbar(p)




In [ ]:
gg = np.abs(blinear-res)

In [ ]:
import matplotlib.pyplot as plt

import cartopy.crs as ccrs

proj=ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=(12, 8))

ax = plt.axes(projection=proj)

h = ax.contourf(plons[3:-3,3:-3],plats[3:-3,3:-3],gg[3:-3,3:-3], transform=proj);

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')

divider = make_axes_locatable(ax)
ax_cb = divider.new_horizontal(size="5%", pad=0.5, axes_class=plt.Axes)


fig.add_axes(ax_cb)
plt.colorbar(h, cax=ax_cb)
#ax.set_global(); 
ax.coastlines();

###### Check with the processed files from GDACS

In [ ]:
pr = xr.open_dataset('/Users/brey/Downloads/HNMS/JRC_0701_00_000.nc')

In [ ]:
prlons = pr.longitude.values
prlats = pr.latitude.values
prvalues = pr.u10.values[0,0,:,:]

###### Combined plot

In [ ]:
xv=[q,blinear,prvalues]
xp=[elon,plons,prlons]
yp=[elat,plats,prlats]

In [ ]:
blinear.shape, prvalues.shape

In [ ]:
plons[0,1]-plons[0,0], prlons[1]-prlons[0]

In [ ]:
plats[1,0]-plats[0,0], prlats[1]-prlats[0]

In [ ]:
import matplotlib.pyplot as plt

import cartopy.crs as ccrs

from mpl_toolkits.axes_grid1 import make_axes_locatable
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from mpl_toolkits.axes_grid1 import AxesGrid

projection = ccrs.PlateCarree()
axes_class = (GeoAxes,
                  dict(map_projection=projection))

fig = plt.figure(figsize=(12,8))
axgr = AxesGrid(fig, 111, axes_class=axes_class,
                    nrows_ncols=(1, 3),
                    axes_pad=1,
                    cbar_location='right',
                    cbar_mode='single',
                    cbar_pad=0.5,
                    cbar_size='10%',
                    label_mode='')  # note the empty label_mode


for i, ax in enumerate(axgr):
        ax.coastlines()
        ax.set_xticks(np.linspace(-180, 180, 5), crs=projection)
        ax.set_yticks(np.linspace(-90, 90, 5), crs=projection)
        lon_formatter = LongitudeFormatter(zero_direction_label=True)
        lat_formatter = LatitudeFormatter()
        ax.xaxis.set_major_formatter(lon_formatter)
        ax.yaxis.set_major_formatter(lat_formatter)

        p = ax.contourf(xp[i], yp[i],xv[i],
                        transform=projection)#,
                     #   cmap='RdBu')

axgr.cbar_axes[0].colorbar(p)




#### compare Nearest with Bylinear

In [ ]:
df = np.abs(blinear-near)

In [ ]:
dfm = np.ma.masked_array(df,np.abs(df)>3)

In [ ]:
np.argwhere(dfm==dfm.max())

In [ ]:
plt.figure()
plt.contour(np.flipud(dfm))
plt.colorbar()

In [ ]:
import matplotlib.pyplot as plt

import cartopy.crs as ccrs

proj=ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=(12, 8))

ax = plt.axes(projection=proj)

h = ax.contourf(plons,plats,dfm, transform=proj);

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')

divider = make_axes_locatable(ax)
ax_cb = divider.new_horizontal(size="5%", pad=0.5, axes_class=plt.Axes)


fig.add_axes(ax_cb)
plt.colorbar(h, cax=ax_cb)
#ax.set_global(); 
ax.coastlines();

## compare  with processed

In [ ]:
nj, ni = pr.u10.shape[2:]

In [ ]:
pr.longitude.min(), pr.longitude.max()

In [ ]:
pr.latitude.min(), pr.latitude.max()

In [ ]:
import pyproj
prj = pyproj.Proj('+proj=eqc +lat_ts=0 +lat_0=0 +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs')
[[a0,a1],[a2,a3]] = prj([-5.2, 30.], [36.613, 46.813])


In [ ]:
area_id = 'HNMS'
description = 'HNMS COSMO'
proj_id = 'eqc'
projection = '+proj=eqc +lat_ts=0 +lat_0=0 +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m'
x_size = ni
y_size = nj
area_extent = (a0, a1, a2, a3)
target_def = utils.get_area_def(area_id, description, proj_id, projection,
                                       x_size, y_size, area_extent)
print(target_def)


In [ ]:
plons,plats = geometry.AreaDefinition.get_lonlats(target_def)

In [ ]:
blinear = bilinear.resample_bilinear(q, orig, target_def,
                                      radius=50e3, neighbours=32,
                                      nprocs=1, fill_value=0,
                                      reduce_data=False, segments=None,
                                      epsilon=0)

In [ ]:
xv=[q,blinear,prvalues]
xp=[elon,plons,prlons]
yp=[elat,plats,prlats]

In [ ]:
import matplotlib.pyplot as plt

import cartopy.crs as ccrs

from mpl_toolkits.axes_grid1 import make_axes_locatable
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from mpl_toolkits.axes_grid1 import AxesGrid

projection = ccrs.PlateCarree()
axes_class = (GeoAxes,
                  dict(map_projection=projection))

fig = plt.figure(figsize=(12,8))
axgr = AxesGrid(fig, 111, axes_class=axes_class,
                    nrows_ncols=(1, 3),
                    axes_pad=1,
                    cbar_location='right',
                    cbar_mode='single',
                    cbar_pad=0.5,
                    cbar_size='10%',
                    label_mode='')  # note the empty label_mode


for i, ax in enumerate(axgr):
        ax.coastlines()
        ax.set_xticks(np.linspace(-180, 180, 5), crs=projection)
        ax.set_yticks(np.linspace(-90, 90, 5), crs=projection)
        lon_formatter = LongitudeFormatter(zero_direction_label=True)
        lat_formatter = LatitudeFormatter()
        ax.xaxis.set_major_formatter(lon_formatter)
        ax.yaxis.set_major_formatter(lat_formatter)

        p = ax.contourf(xp[i], yp[i],xv[i],
                        transform=projection)#,
                     #   cmap='RdBu')

axgr.cbar_axes[0].colorbar(p)




In [ ]:
df = np.abs(np.flipud(blinear)-pr.u10[0,0,:,:].values)

In [ ]:
df.max()

In [ ]:
dfm = np.ma.masked_array(df,np.abs(df)<1)

In [ ]:
np.argwhere(dfm==dfm.max())

In [ ]:
plt.figure()
plt.contourf(np.flipud(df))
plt.colorbar()

In [ ]:
import matplotlib.pyplot as plt

import cartopy.crs as ccrs

proj=ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=(12, 8))

ax = plt.axes(projection=proj)

h = ax.contourf(plons,plats,df, transform=proj);

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')

divider = make_axes_locatable(ax)
ax_cb = divider.new_horizontal(size="5%", pad=0.5, axes_class=plt.Axes)


fig.add_axes(ax_cb)
plt.colorbar(h, cax=ax_cb)
#ax.set_global(); 
ax.coastlines();

#### compare Nearest with processed

In [ ]:
#kd_tree.resample_nearest(orig,q,targ,radius_of_influence=50000,fill_value=999999)
near = kd_tree.resample_nearest(orig, q, target_def, radius_of_influence=50000, epsilon=0.5)

In [ ]:
df = np.abs(np.flipud(near)-pr.u10[0,0,:,:].values)

In [ ]:
df.max()

In [ ]:
dfm = np.ma.masked_array(df,np.abs(df)<1)

In [ ]:
import matplotlib.pyplot as plt

import cartopy.crs as ccrs

proj=ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=(12, 8))

ax = plt.axes(projection=proj)

h = ax.contourf(plons,plats,df, transform=proj);

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')

divider = make_axes_locatable(ax)
ax_cb = divider.new_horizontal(size="5%", pad=0.5, axes_class=plt.Axes)


fig.add_axes(ax_cb)
plt.colorbar(h, cax=ax_cb)
#ax.set_global(); 
ax.coastlines();